Group Name : Group 6

Student pace: Full time Hybrid

Instructor name:Diana Mongina


**BUSINESS PROBLEM**

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

**Objectives**
